Use only the data after 2023

In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder


In [ ]:
df = pd.read_csv('chicago_crime.csv')

df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Date'].dt.year >= 2023]

Data conversion

In [2]:
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

Handle missing values:\
For longitude and latitude, fill with median of the block or district.
For ward and community area, fill with the most frequent value in the same district.

In [3]:
df['latitude'] = df.groupby('block')['latitude'].transform(lambda x: x.fillna(x.median()))
df['longitude'] = df.groupby('block')['longitude'].transform(lambda x: x.fillna(x.median()))
df['latitude'].fillna(df.groupby('district')['latitude'].transform('median'), inplace=True)
df['longitude'].fillna(df.groupby('district')['longitude'].transform('median'), inplace=True)
df['ward'] = df.groupby('district')['ward'].transform(lambda x: x.fillna(x.mode()[0]))
df['community_area'] = df.groupby('district')['community_area'].transform(lambda x: x.fillna(x.mode()[0]))
df['location_description'].fillna('unknown', inplace=True)

e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
e:\1MCOMP\25spring-sem2\IT5006\.venv\Lib\site-packages\numpy\lib\_nanf

In [4]:
df.isnull().sum()

id                         0
case_number                0
date                       0
block                      0
iucr                       0
primary_type               0
description                0
location_description       0
arrest                     0
domestic                   0
beat                       0
district                   0
ward                       0
community_area             0
fbi_code                   0
x_coordinate            1376
y_coordinate            1376
year                       0
updated_on                 0
latitude                   0
longitude                  0
location                1376
dtype: int64

Transform date to datetime format and extract other date-related data

In [5]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%dT%H:%M:%S.%f')
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['weekday'] = df['date'].dt.weekday
df['is_weekend'] = (df['weekday'] >= 5).astype(int)

Aggregate for spatial data

In [6]:
crime_counts_block = df['block'].value_counts().to_dict()
crime_counts_district = df['district'].value_counts().to_dict()
df['crime_count_block'] = df['block'].map(crime_counts_block)
df['crime_count_district'] = df['district'].map(crime_counts_district)

Transform boolean data to integer

In [7]:
df['arrest'] = df['arrest'].astype(int)
df['domestic'] = df['domestic'].astype(int)

In [ ]:
df.head()

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,latitude,longitude,location,month,day,hour,weekday,is_weekend,crime_count_block,crime_count_district
0,13770762,JJ175924,2025-03-08,064XX S DR MARTIN LUTHER KING JR DR,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,0,1,...,41.777838,-87.615566,"{'latitude': '41.777838294', 'longitude': '-87...",3,8,0,5,1,393,25155
1,13769396,JJ174256,2025-03-08,051XX N SHERIDAN RD,0810,THEFT,OVER $500,SMALL RETAIL STORE,0,1,...,41.975729,-87.654990,"{'latitude': '41.975729415', 'longitude': '-87...",3,8,0,5,1,43,10810
2,13768651,JJ173314,2025-03-08,040XX W 21ST PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,0,1,...,41.852572,-87.725713,"{'latitude': '41.852571947', 'longitude': '-87...",3,8,0,5,1,36,20723
3,13769096,JJ174022,2025-03-08,130XX S DR MARTIN LUTHER KING JR DR,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,RESIDENCE,0,0,...,41.658159,-87.613497,"{'latitude': '41.658159346', 'longitude': '-87...",3,8,0,5,1,57,19959
4,13769193,JJ174175,2025-03-08,031XX N BROADWAY,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,0,0,...,41.939596,-87.644387,"{'latitude': '41.939596414', 'longitude': '-87...",3,8,0,5,1,122,26892


One-hot encode for location_description, after redundancy removal and other handling.

In [8]:
top_categories = df['location_description'].value_counts()[df['location_description'].value_counts() >= 60].index
df['location_description'] = df['location_description'].apply(lambda x: x if x in top_categories else 'others')

In [9]:
df['location_description'] = df['location_description'].str.strip().str.lower()

In [12]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_features = encoder.fit_transform(df[['location_description']])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['location_description']))

df = pd.concat([df.drop('location_description', axis=1), encoded_df], axis=1)


Drop unneccessary columns

In [13]:
to_drop = [
    'id', 'case_number', 'block', 'primary_type', 'description',
    'x_coordinate', 'y_coordinate', 'updated_on', 'location'
]
df.drop(columns=to_drop, inplace=True)

Infer primary type and description by iucr

In [14]:
df['primary_type'] = df['iucr'].str[:2]
df['description'] = df['iucr'].str[2:]

df = df[df['description'].str.match(r'^\d+$')]

In [15]:
df.head()

,date,iucr,arrest,domestic,beat,district,ward,community_area,fbi_code,year,...,location_description_taxicab,location_description_unknown,location_description_vacant lot / land,location_description_vehicle - commercial,location_description_vehicle - delivery truck,"location_description_vehicle - other ride share service (lyft, uber, etc.)",location_description_vehicle non-commercial,location_description_warehouse,primary_type,description
0,2025-03-08,1754,0,1,0312,003,20,69,02,2025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17,54
1,2025-03-08,0810,0,1,2024,020,48,3,06,2025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,08,10
2,2025-03-08,0486,0,1,1012,010,24,29,08B,2025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,04,86
3,2025-03-08,2825,0,0,0533,005,9,54,26,2025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28,25
4,2025-03-08,1320,0,0,1934,019,44,6,14,2025,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,20
